In [1]:
# importamos librerias necesarias
import pandas as pd
import numpy as np

In [3]:
# leemos los datos de la EAC
eac = pd.read_csv("../data/tr_eac_comercio_al_por_menor_2013_2021.csv", usecols=["CODIGO_ACTIVIDAD", "ANIO", "M100A", "M310A", "M971A",
"M200A", "M500A", "M999A"])

eac_2022 = pd.read_csv("../data/tr_eac_comercio_al_por_menor_2021_2022.csv", usecols=["CODIGO_ACTIVIDAD", "ANIO", "M100A", "M971A",
"M200A", "M500A", "M999A"])

eac = pd.concat([eac, eac_2022], ignore_index=True).fillna(0)
eac["M310A"] = eac["M310A"].astype("int64")

In [4]:
eac

,CODIGO_ACTIVIDAD,ANIO,M100A,M310A,M971A,M200A,M500A,M999A
0,46,2013,4834404708,1572786,155020457,13240140,6824255,29573435
1,461,2013,865475807,340456,2988274,238869,985099,1241286
2,4611,2013,819161921,309850,2920010,234035,965384,948126
3,4612,2013,46313886,30606,68264,4834,19715,293160
4,462,2013,1087797323,520173,1699730,4624706,3752297,15096024
...,...,...,...,...,...,...,...,...
315,4682,2022,330505889,0,218054,3078063,208995,787504
316,4683,2022,70732355,0,79734,1008842,66427,68919
317,4684,2022,1529003818,0,48563,2462790,1195338,3846598
318,469,2022,46056216,0,5261755,4253174,0,1824839


In [5]:
# Definimos los ingresos por categoría
eac['ingresos'] = eac[['M100A', 'M310A', 'M971A', 'M200A', 'M500A', 'M999A']].sum(axis=1)

In [6]:
eac.head()

,CODIGO_ACTIVIDAD,ANIO,M100A,M310A,M971A,M200A,M500A,M999A,ingresos
0,46,2013,4834404708,1572786,155020457,13240140,6824255,29573435,5040635781
1,461,2013,865475807,340456,2988274,238869,985099,1241286,871269791
2,4611,2013,819161921,309850,2920010,234035,965384,948126,824539326
3,4612,2013,46313886,30606,68264,4834,19715,293160,46730465
4,462,2013,1087797323,520173,1699730,4624706,3752297,15096024,1113490253


In [13]:
# sacamos los ingresos totales anuales
eac_totales = eac.groupby(['ANIO'])['ingresos'].sum().reset_index()

In [14]:
eac_totales

,ANIO,ingresos
0,2013,15121907343
1,2014,15515291085
2,2015,16164883533
3,2016,17089669110
4,2017,18452245812
5,2018,19668107310
6,2019,22355266467
7,2020,20920932453
8,2021,24250325880
9,2022,26068243707


In [15]:
# Filtramos para las actividades que nos interesan
eac_filtrado = eac[eac['CODIGO_ACTIVIDAD'].isin([4621, 4622, 463, 4641, 4659, 4661, 4682, 4612, 4652, 4651, 466, 4671, 4653])]

In [16]:
# Mapeamos las categorías de la EAC a las categorías de la ANTAD
codigo_mapping = {
    4622: 'departamental',
    4621: 'autoservicio'
}
eac_filtrado
eac_filtrado['tipo_antad'] = eac_filtrado['CODIGO_ACTIVIDAD'].map(codigo_mapping).fillna('especializada')

/var/folders/_n/15dj3g9532b7lqmq3dcy_1jw0000gp/T/ipykernel_95752/961873237.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eac_filtrado['tipo_antad'] = eac_filtrado['CODIGO_ACTIVIDAD'].map(codigo_mapping).fillna('especializada')


In [11]:
# Sacamos el ingreso por cada tipo de tienda ANTAD
eac_antad = eac_filtrado.groupby(['ANIO', 'tipo_antad'])['ingresos'].sum().reset_index()

In [12]:
eac_antad

,ANIO,tipo_antad,ingresos
0,2013,autoservicio,889863079
1,2013,departamental,223627174
2,2013,especializada,2168761481
3,2014,autoservicio,943465864
4,2014,departamental,242787312
5,2014,especializada,2301474533
6,2015,autoservicio,1029486387
7,2015,departamental,270920695
8,2015,especializada,2309270990
9,2016,autoservicio,1112828502


In [17]:
# Sacamos el porcentaje de participacion de cada tipo de tienda del total
eac_antad['participacion'] = eac_antad.apply(
    lambda row: (row['ingresos'] / eac_totales[eac_totales['ANIO'] == row['ANIO']]['ingresos'].iloc[0]), 
    axis=1
)

In [18]:
eac_antad

,ANIO,tipo_antad,ingresos,participacion
0,2013,autoservicio,889863079,0.058846
1,2013,departamental,223627174,0.014788
2,2013,especializada,2168761481,0.143419
3,2014,autoservicio,943465864,0.060809
4,2014,departamental,242787312,0.015648
5,2014,especializada,2301474533,0.148336
6,2015,autoservicio,1029486387,0.063687
7,2015,departamental,270920695,0.016760
8,2015,especializada,2309270990,0.142857
9,2016,autoservicio,1112828502,0.065117


In [21]:
# Leemos el excel de valor agregado al comercio electronico 
valor_agregado = pd.read_excel("../data/VABCOEL_3.xlsx", skiprows=4, nrows=6)

In [22]:
valor_agregado

,Concepto,2013,2014,2015,2016,2017,2018,2019,2020,2021R,2022P
0,Millones de pesos a precios de 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Valor agregado bruto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total,656056.252,668374.405,776955.053,863448.187,942764.771,1004368.757,1025423.135,1064261.695,1176919.328,1610984.741
3,Comercio al por mayor,146567.676,141049.624,184849.208,225100.204,244874.067,254101.158,226970.120,276443.481,280796.606,360634.978
4,Comercio al por menor,67925.163,62970.162,89483.787,107195.119,126718.741,136018.250,309188.707,379362.524,332874.178,425434.981
5,______aOtros servicios,441563.413,464354.619,502622.058,531152.864,571171.963,614249.349,489264.308,408455.690,563248.544,824914.782


In [23]:
valor_agregado.Concepto = valor_agregado.Concepto.apply(str.lstrip)

In [24]:
va_com_menor = valor_agregado[valor_agregado["Concepto"]=="Comercio al por menor"]

In [25]:
va_com_menor

,Concepto,2013,2014,2015,2016,2017,2018,2019,2020,2021R,2022P
4,Comercio al por menor,67925.163,62970.162,89483.787,107195.119,126718.741,136018.25,309188.707,379362.524,332874.178,425434.981


In [26]:
va_com_menor = va_com_menor.melt(
    id_vars=['Concepto'],  
    var_name='ANIO',       
    value_name='valor_agregado' 
)

va_com_menor['ANIO'] = va_com_menor['ANIO'].str.replace('R', '').str.replace('P', '')

In [27]:
va_com_menor["ANIO"] = va_com_menor["ANIO"].astype("int64")

In [28]:
va_com_menor.dtypes

Concepto           object
ANIO                int64
valor_agregado    float64
dtype: object

In [29]:
# Agregamos el valor agregado del comercio electronico para cada año y lo multiplicamos por la participación
eac_antad['valor_agregado'] = eac_antad.apply(
    lambda row: (row['participacion'] * va_com_menor[va_com_menor['ANIO'] == row['ANIO']]['valor_agregado'].iloc[0]), 
    axis=1
)

In [30]:
eac_antad

,ANIO,tipo_antad,ingresos,participacion,valor_agregado
0,2013,autoservicio,889863079,0.058846,3997.121085
1,2013,departamental,223627174,0.014788,1004.497111
2,2013,especializada,2168761481,0.143419,9741.725945
3,2014,autoservicio,943465864,0.060809,3829.138491
4,2014,departamental,242787312,0.015648,985.373480
5,2014,especializada,2301474533,0.148336,9340.735110
6,2015,autoservicio,1029486387,0.063687,5698.917681
7,2015,departamental,270920695,0.016760,1499.733030
8,2015,especializada,2309270990,0.142857,12783.408737
9,2016,autoservicio,1112828502,0.065117,6980.227817


In [32]:
# Graficamos los resultados
import plotly.express as px

fig = px.line(eac_antad, 
              x='ANIO', 
              y='valor_agregado',
              color='tipo_antad',
              markers=True,
              title='Valor agregado del comercio electrónico para cada tipo de tiendas de la ANTAD de 2013 a 2022')

fig.update_layout(
    xaxis_title='Año',
    yaxis_title='Valor Agregado',
    legend_title='Tipo de tienda'
)

fig.show()